In [127]:
import os
import shutil
import numpy as np

In [3]:
import parmed as pmd

In [138]:
import paprika
from paprika import align
from paprika import dummy
from paprika import tleap

from paprika.restraints import static_DAT_restraint
from paprika.restraints import DAT_restraint

from paprika.tleap import System

# pAPRika

Here is a basic outline of how to perform a host-guest binding free energy calculation using pAPRika as an interface for the attach-pull-release workflow.

To run a calculation, we need a few things:
- Structure of the host and guest,
- Separate `mol2` files for the host and guest,
- Any `frcmod` files that have non-standard force field parameters,
- Three atoms on the host and two atoms on the guest, to act as anchors.

## Prepare the host-guest complex by aligning and adding dummy atoms.

In [14]:
structure = pmd.load_file("complex/a-coc-p.pdb", structure=True)

We are going to align the structure so that the two guest atoms are along the z axis.

In [15]:
structure = align.zalign(system, ":COC@C1", ":COC@O1", save=True, filename="complex/aligned.pdb")

/home/dslochower/hgst-3tb-data/projects/pAPRika/paprika/align.py:31: RuntimeWarning: invalid value encountered in true_divide
  x     = np.cross(mask2_com, axis) / np.linalg.norm(np.cross(mask2_com, axis))


In [16]:
structure = pmd.load_file("complex/aligned.pdb", structure=True)

In [17]:
structure = dummy.add_dummy(system, residue_name="DM1", z=-6.000)
structure = dummy.add_dummy(system, residue_name="DM2", z=-9.000)
structure = dummy.add_dummy(system, residue_name="DM3", z=-11.200, y=2.200)

In [18]:
structure.write_pdb("complex/aligned_with_dummy.pdb")

![](images/aligned_with_dummy_top.png)

![](images/aligned_with_dummy_side.png)

# Create AMBER parameter files for the aligned system

First, we write a `mol2` file and `frmod` for the dummy atoms. Then, we write a template for `tleap` to create a `prmtop` and `inpcrd` for the structure using the relevant `leaprc` files.

In [22]:
dummy.write_dummy_frcmod(filepath="complex/dummy.frcmod")

In [24]:
dummy.write_dummy_mol2(residue_name="DM1", filepath="complex/dm1.mol2")
dummy.write_dummy_mol2(residue_name="DM2", filepath="complex/dm2.mol2")
dummy.write_dummy_mol2(residue_name="DM3", filepath="complex/dm3.mol2")

At the moment, it is easier to build the complex with `tleap` outside of `paprika`, inside the `complex` directory.

In [4]:
! cat complex/build-complex.tleap.in


source CD-leaprc
source leaprc.gaff
loadamberparams dummy.frcmod
COC = loadmol2 coc.mol2
DM1 = loadmol2 dm1.mol2
DM2 = loadmol2 dm2.mol2
DM3 = loadmol2 dm3.mol2
complex = loadpdb aligned_with_dummy.pdb

desc complex
saveamberparm complex aligned_with_dummy.prmtop aligned_with_dummy.rst7

quit


First, we we will add the restraints. Based on the restraints, pAPRika will determine the number of windows in each phase of the calculation. We will then create the necessary directories, and solvate each structure separately. 

## Restraints

In this calculation, I will setup four types of restraints.
- Static restraints: these six restraints keep the host and in the proper orientation during the simulation (**necessary**),
- Guest restraints: these restraints pull the guest away from the host along the z axis (**necessary**),
- Conformational restraints: these restraints alter the conformational sampling of the host molecule (**optional**), and
- Wall restraints: these restraints help define the bound state of the guest (**optional**).

More details on the can be found in the Computational Calorimetry paper. The first second static restraint limits the distance between the dummy atoms and the host. The second state restraint, restrains the angle the host atom can make with two dummy atoms. And so on.


      .DM3                 ####H2
           .DM2   .DM1     H1    G1####G2   -----> z-axis
                           ####H3



To facilitate setting up the restraints, I'm going to make a shorthand `dict` for the anchor atoms. The host anchor atoms should be heavy atoms that are distributed around the binding cavity. The guest anchor atoms should be heavy atoms that are aligned along the $z$ axis.

In [106]:
    anchor_atoms = {
        "D1": f":DM1",
        "D2": f":DM2",
        "D3": f":DM3",
        "H1": f":1@O3",
        "H2": f":3@C1",
        "H3": f":5@C6",
        "G1": f":7@C1",
        "G2": f":7@O1",
    }


In [107]:
structure = pmd.load_file("complex/aligned_with_dummy.prmtop",
                    "complex/aligned_with_dummy.rst7",
                    structure = True)

In [108]:
def setup_static_restraints(
    anchor_atoms, windows, structure, distance_fc=5.0, angle_fc=100.0
):
    static_restraints = []
    static_restraint_atoms = [
        [anchor_atoms["D1"], anchor_atoms["H1"]],
        [anchor_atoms["D2"], anchor_atoms["D1"], anchor_atoms["H1"]],
        [anchor_atoms["D1"], anchor_atoms["H1"], anchor_atoms["H2"]],
        [
            anchor_atoms["D3"],
            anchor_atoms["D2"],
            anchor_atoms["D1"],
            anchor_atoms["H1"],
        ],
        [
            anchor_atoms["D2"],
            anchor_atoms["D1"],
            anchor_atoms["H1"],
            anchor_atoms["H2"],
        ],
        [
            anchor_atoms["D1"],
            anchor_atoms["H1"],
            anchor_atoms["H2"],
            anchor_atoms["H3"],
        ],
    ]

    for index, atoms in enumerate(static_restraint_atoms):
        if len(atoms) > 2:
            fc = angle_fc
        else:
            fc = distance_fc
        print(f"Applying static restraint on atoms {atoms} with force constant = {fc} (units depend on whether this is angle or distane.)")
        
        this = static_DAT_restraint(
            restraint_mask_list=atoms,
            num_window_list=windows,
            ref_structure=structure,
            force_constant=fc,
            amber_index=True,
        )

        static_restraints.append(this)
    return static_restraints


I'm short in time so I'm skipping the commentary.

Next are the guest restraints. These do the "pulling."

In [109]:
def setup_guest_restraints(
    anchor_atoms,
    windows,
    structure,
    distance_fc=5.0,
    angle_fc=100.0,
    pull_initial=6.0,
    pull_final=24.0,
):
    guest_restraints = []

    guest_restraint_atoms = [
        [anchor_atoms["D1"], anchor_atoms["G1"]],
        [anchor_atoms["G2"], anchor_atoms["D1"], anchor_atoms["G1"]],
        [anchor_atoms["D1"], anchor_atoms["G1"], anchor_atoms["G2"]],
    ]
    guest_restraint_targets = {
        "initial": [pull_initial, 180.0, 180.0],
        "final": [pull_final, 180.0, 180.0],
    }

    for index, atoms in enumerate(guest_restraint_atoms):
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = structure
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach["fc_final"] = angle_fc
            this.release["fc_final"] = angle_fc
        else:
            this.attach["fc_final"] = distance_fc
            this.release["fc_final"] = angle_fc
        this.attach["target"] = guest_restraint_targets["initial"][index]
        this.attach["fraction_list"] = attach_fractions

        this.pull["target_final"] = guest_restraint_targets["final"][index]
        this.pull["num_windows"] = windows[1]
        
        this.release['target'] = guest_restraint_targets["final"][index]
        # Keep the guest restraints on during release.
        this.release['fraction_list'] = [1.0] * windows[2]
    
        print(f"Applying guest restraint between atoms {atoms}")
        this.initialize()

        guest_restraints.append(this)
    return guest_restraints


In [110]:
def setup_conformation_restraints(
    template, targets, windows, attach_fractions, structure, resname, fc=6.0
):

    conformational_restraints = []
    host_residues = len(structure[":{}".format(resname.upper())].residues)
    first_host_residue = structure[":{}".format(resname.upper())].residues[0].number + 1

    for n in range(first_host_residue, host_residues + first_host_residue):
        if n + 1 < host_residues + first_host_residue:
            next_residue = n + 1
        else:
            next_residue = first_host_residue

        for (index, atoms), target in zip(enumerate(template), targets):
            
            conformational_restraint_atoms = []
            if index == 0:
                conformational_restraint_atoms.append(f":{n}@{atoms[0]}")
                conformational_restraint_atoms.append(f":{n}@{atoms[1]}")
                conformational_restraint_atoms.append(f":{n}@{atoms[2]}")
                conformational_restraint_atoms.append(f":{next_residue}@{atoms[3]}")
            else:
                conformational_restraint_atoms.append(f":{n}@{atoms[0]}")
                conformational_restraint_atoms.append(f":{n}@{atoms[1]}")
                conformational_restraint_atoms.append(f":{next_residue}@{atoms[2]}")
                conformational_restraint_atoms.append(f":{next_residue}@{atoms[3]}")

            this = DAT_restraint()
            this.auto_apr = True
            this.amber_index = True
            this.topology = structure
            this.mask1 = conformational_restraint_atoms[0]
            this.mask2 = conformational_restraint_atoms[1]
            this.mask3 = conformational_restraint_atoms[2]
            this.mask4 = conformational_restraint_atoms[3]

            this.attach["fraction_list"] = attach_fractions
            this.attach["target"] = target
            this.attach["fc_final"] = fc
            this.pull["target_final"] = target
            this.pull["num_windows"] = windows[1]
            
            this.release["fraction_list"] = attach_fractions[::-1]
            this.release["target"] = target
            this.release["fc_final"] = fc

            print(f"Applying guest restraint between atoms {conformational_restraint_atoms}")
            this.initialize()
            conformational_restraints.append(this)
    return conformational_restraints


In [111]:
def setup_guest_wall_restraints(template, targets, structure, resname, angle_fc=500.0, distance_fc=50.0):

    guest_wall_restraints = []
    host_residues = len(structure[":{}".format(resname.upper())].residues)
    first_host_residue = structure[":{}".format(resname.upper())].residues[0].number + 1

    for n in range(first_host_residue, host_residues + first_host_residue):

        for (index, atoms), target in zip(enumerate(template), targets):
            guest_wall_restraint_atoms = []
            if index == 0 or index == 1:
                guest_wall_restraint_atoms.append(f":{n}@{atoms[0]}")
                guest_wall_restraint_atoms.append(f"{atoms[1]}")
                angle = False
            else:
                guest_wall_restraint_atoms.append(f"{atoms[0]}")
                guest_wall_restraint_atoms.append(f"{atoms[1]}")
                guest_wall_restraint_atoms.append(f"{atoms[2]}")
                angle = True
                
                
            this = DAT_restraint()
            this.auto_apr = True
            this.amber_index = True
            this.topology = structure
            this.mask1 = guest_wall_restraint_atoms[0]
            this.mask2 = guest_wall_restraint_atoms[1]
            if angle:
                this.mask3 = guest_wall_restraint_atoms[2]
                this.attach["fc_initial"] = angle_fc
                this.attach["fc_final"] = angle_fc
                this.custom_restraint_values["rk2"] = 500.0
                this.custom_restraint_values["rk3"] = 0.0
            else:
                this.attach["fc_initial"] = distance_fc
                this.attach["fc_final"] = distance_fc
                this.custom_restraint_values["rk2"] = 50.0
                this.custom_restraint_values["rk3"] = 50.0
                this.custom_restraint_values["r1"] = 0.0
                this.custom_restraint_values["r2"] = 0.0

            this.attach["target"] = target
            this.attach["num_windows"] = windows[1]

            if angle:
                print(f"Applying flat-bottom restraint between 0 and {target}")
            else:
                print(f"Applying flat-bottom restraint between {this.custom_restraint_values['r2']} and {target}")

            this.initialize()
            guest_wall_restraints.append(this)
    return guest_wall_restraints


In [112]:
attach_string = "0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00"
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = "0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 16.80 17.20 17.60 18.00"
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

release_fractions = attach_fractions[::-1]

windows = [len(attach_fractions), len(pull_distances), len(release_fractions)]
print(f"There are {windows} windows in this attach-pull-release calculation.")

There are [15, 46, 15] windows in this attach-pull-release calculation.


In [113]:
static_restraints = setup_static_restraints(
    anchor_atoms, windows, structure, distance_fc=5.0, angle_fc=100.0
)

Applying static restraint on atoms [':DM1', ':1@O3'] with force constant = 5.0 (units depend on whether this is angle or distane.)
Applying static restraint on atoms [':DM2', ':DM1', ':1@O3'] with force constant = 100.0 (units depend on whether this is angle or distane.)
Applying static restraint on atoms [':DM1', ':1@O3', ':3@C1'] with force constant = 100.0 (units depend on whether this is angle or distane.)
Applying static restraint on atoms [':DM3', ':DM2', ':DM1', ':1@O3'] with force constant = 100.0 (units depend on whether this is angle or distane.)
Applying static restraint on atoms [':DM2', ':DM1', ':1@O3', ':3@C1'] with force constant = 100.0 (units depend on whether this is angle or distane.)
Applying static restraint on atoms [':DM1', ':1@O3', ':3@C1', ':5@C6'] with force constant = 100.0 (units depend on whether this is angle or distane.)


In [114]:
guest_restraints = setup_guest_restraints(
    anchor_atoms,
    windows,
    structure,
    distance_fc=5.0,
    angle_fc=100.0,
    pull_initial=6.0,
    pull_final=24.0,
)


Applying guest restraint between atoms [':DM1', ':7@C1']
Applying guest restraint between atoms [':7@O1', ':DM1', ':7@C1']
Applying guest restraint between atoms [':DM1', ':7@C1', ':7@O1']


In [115]:
host_conformational_template = [["O5", "C1", "O1", "C4"], ["C1", "O1", "C4", "C5"]]
host_conformational_targets = [104.30, -108.8]

In [116]:
conformational_restraints = setup_conformation_restraints(
    host_conformational_template,
    host_conformational_targets,
    windows,
    attach_fractions,
    structure,
    resname="MGO",
    fc=6.0,
)

Applying guest restraint between atoms [':1@O5', ':1@C1', ':1@O1', ':2@C4']
Applying guest restraint between atoms [':1@C1', ':1@O1', ':2@C4', ':2@C5']
Applying guest restraint between atoms [':2@O5', ':2@C1', ':2@O1', ':3@C4']
Applying guest restraint between atoms [':2@C1', ':2@O1', ':3@C4', ':3@C5']
Applying guest restraint between atoms [':3@O5', ':3@C1', ':3@O1', ':4@C4']
Applying guest restraint between atoms [':3@C1', ':3@O1', ':4@C4', ':4@C5']
Applying guest restraint between atoms [':4@O5', ':4@C1', ':4@O1', ':5@C4']
Applying guest restraint between atoms [':4@C1', ':4@O1', ':5@C4', ':5@C5']
Applying guest restraint between atoms [':5@O5', ':5@C1', ':5@O1', ':6@C4']
Applying guest restraint between atoms [':5@C1', ':5@O1', ':6@C4', ':6@C5']
Applying guest restraint between atoms [':6@O5', ':6@C1', ':6@O1', ':1@C4']
Applying guest restraint between atoms [':6@C1', ':6@O1', ':1@C4', ':1@C5']


In [117]:
guest_wall_template = [
    ["O2", anchor_atoms["G1"]],
    ["O6", anchor_atoms["G1"]],
    [anchor_atoms["D2"], anchor_atoms["G1"], anchor_atoms["G2"]],
]
guest_wall_targets = [11.3, 13.3, 80.0]

In [118]:
guest_wall_restraints = setup_guest_wall_restraints(
    guest_wall_template,
    guest_wall_targets,
    structure,
    resname="MGO",
    angle_fc=500.0,
    distance_fc=50.0,
)

Applying flat-bottom restraint between 0.0 and 11.3
Applying flat-bottom restraint between 0.0 and 13.3
Applying flat-bottom restraint between 0 and 80.0
Applying flat-bottom restraint between 0.0 and 11.3
Applying flat-bottom restraint between 0.0 and 13.3
Applying flat-bottom restraint between 0 and 80.0
Applying flat-bottom restraint between 0.0 and 11.3
Applying flat-bottom restraint between 0.0 and 13.3
Applying flat-bottom restraint between 0 and 80.0
Applying flat-bottom restraint between 0.0 and 11.3
Applying flat-bottom restraint between 0.0 and 13.3
Applying flat-bottom restraint between 0 and 80.0
Applying flat-bottom restraint between 0.0 and 11.3
Applying flat-bottom restraint between 0.0 and 13.3
Applying flat-bottom restraint between 0 and 80.0
Applying flat-bottom restraint between 0.0 and 11.3
Applying flat-bottom restraint between 0.0 and 13.3
Applying flat-bottom restraint between 0 and 80.0



After the restraints are applied, we create the window list (which checks the the restraints are consistent with themselves) and then create the directories for each umbrella sampling window.

In [119]:
from paprika.restraints import create_window_list
from paprika.restraints import amber_restraint_line
from paprika.utils import make_window_dirs

In [120]:
window_list = create_window_list(guest_restraints)
for window in window_list:
    if not os.path.isdir(os.path.join("windows", window)):
        print(f'Creating {os.path.join("windows", window)}')
        os.makedirs(os.path.join("windows", window))


Next, we write the restraint information in each window

In [122]:
attach_windows = [i for i in window_list if i[0] == "a"]
pull_windows = [i for i in window_list if i[0] == "p"]
release_windows = [i for i in window_list if i[0] == "r"]
    
for phase, phase_windows in zip(["attach", "pull", "release"],
                                [attach_windows, pull_windows, release_windows]):
    
    if phase == "attach":
        restraints = (
        static_restraints
        + conformational_restraints
        + guest_restraints
        + guest_wall_restraints
        )
    else:
        restraints = (
        static_restraints
        + conformational_restraints
        + guest_restraints
        )

    for window in phase_windows:
        restraints_file = os.path.join("windows", window, "disang.rest")
        with open(restraints_file, "a") as file:
            for restraint in restraints:
                string = amber_restraint_line(restraint, phase, int(window[1:]))
                if string is not None:
                    file.write(string)

## Create the structure in each window

Attach windows get the starting structure.
We translate the guest for the pull windows.
Release windows get the final pull structure.

In [136]:
for window in attach_windows:
    shutil.copy(os.path.join("complex", "aligned_with_dummy.prmtop"),
                os.path.join("windows", window, "aligned_with_dummy.prmtop"))

    shutil.copy(os.path.join("complex", "aligned_with_dummy.rst7"),
            os.path.join("windows", window, "aligned_with_dummy.rst7"))
    shutil.copy(os.path.join("complex", "aligned_with_dummy.pdb"),
        os.path.join("windows", window, "aligned_with_dummy.pdb"))

for window in pull_windows:
    structure = pmd.load_file("complex/aligned_with_dummy.prmtop",
                             "complex/aligned_with_dummy.rst7",
                             structure=True)
    target_difference = guest_restraints[0].phase['pull']['targets'][int(window[1:])] - \
                        guest_restraints[0].pull['target_initial']

    print(f'{window}: Translating by {target_difference:03f} A...')
    for atom in structure.atoms:
        if atom.residue.name == 'COC':
            atom.xz += target_difference
            
    structure.save(os.path.join("windows", window, "aligned_with_dummy.rst7"))
    structure.save(os.path.join("windows", window, "aligned_with_dummy.pdb"))
    shutil.copy(os.path.join("complex", "aligned_with_dummy.prmtop"),
        os.path.join("windows", window, "aligned_with_dummy.prmtop"))


for window in release_windows:
    shutil.copy(os.path.join("windows", pull_windows[-1], "aligned_with_dummy.rst7"),
        os.path.join("windows", window, "aligned_with_dummy.rst7"))
    shutil.copy(os.path.join("windows", pull_windows[-1], "aligned_with_dummy.pdb"),
        os.path.join("windows", window, "aligned_with_dummy.pdb"))
    shutil.copy(os.path.join("complex", "aligned_with_dummy.prmtop"),
            os.path.join("windows", window, "aligned_with_dummy.prmtop"))

p000: Translating by 0.000000 A...
p001: Translating by 0.400000 A...
p002: Translating by 0.800000 A...
p003: Translating by 1.200000 A...
p004: Translating by 1.600000 A...
p005: Translating by 2.000000 A...
p006: Translating by 2.400000 A...
p007: Translating by 2.800000 A...
p008: Translating by 3.200000 A...
p009: Translating by 3.600000 A...
p010: Translating by 4.000000 A...
p011: Translating by 4.400000 A...
p012: Translating by 4.800000 A...
p013: Translating by 5.200000 A...
p014: Translating by 5.600000 A...
p015: Translating by 6.000000 A...
p016: Translating by 6.400000 A...
p017: Translating by 6.800000 A...
p018: Translating by 7.200000 A...
p019: Translating by 7.600000 A...
p020: Translating by 8.000000 A...
p021: Translating by 8.400000 A...
p022: Translating by 8.800000 A...
p023: Translating by 9.200000 A...
p024: Translating by 9.600000 A...
p025: Translating by 10.000000 A...
p026: Translating by 10.400000 A...
p027: Translating by 10.800000 A...
p028: Translating

## Solvate in each window

This isn't going to work exactly, because of path issues, but the code would be something close to this:

```python
for window_index, window in enumerate(window_list):

    system = System()
    system.output_path = os.path.join('windows', window)        

    system.target_waters = 2000
    system.output_prefix = 'solvated'

    system.neutralize = True
    system.pbc_type = 'rectangular'
    system.add_ions = ['Na+', 4, 'Cl-', 4]
    system.template_lines = ['source leaprc.water.tip3p',
                           'source leaprc.gaff',
                           'loadamberparams .frcmod',
                           'HST = loadmol2 hst.mol2',
                           'loadamberparams gst.frcmod',
                           'GST = loadmol2 gst.mol2',
                           'loadamberparams dummy.frcmod',
                           'DM1 = loadmol2 dm1.mol2',
                           'DM2 = loadmol2 dm2.mol2',
                           'DM3 = loadmol2 dm3.mol2',
                           'model = loadpdb hg.pdb'
    ]

    system.build()
```

## Write a simulation file in each window

This isn't going to work, but the code will look something similar to this.

```python
import os as os
import subprocess as sp
import shutil as shutil
import re as re

import paprika as paprika
from paprika.amber import Simulation

import logging


def recenter(prmtop, inpcrd, mask):
    trajout = inpcrd.split(".")[0] + "-centered.inpcrd"
    cpptraj = f"""
    parm {prmtop}
    trajin {inpcrd}
    center {mask} origin
    trajout {trajout} restart
    """
    with open("center.in", "w") as f:
        f.write(cpptraj)
    sp.call("cpptraj -i center.in > center.out", shell=True)


logging.basicConfig(
    filename="tscc.log",
    format="%(asctime)s %(message)s",
    datefmt="%Y-%m-%d %I:%M:%S %p",
    level=logging.DEBUG,
)
logging.info("Started logging...")
logging.info(paprika.__version__)
hostname = sp.check_output(["hostname"])
nvidia_smi = sp.check_output(["nvidia-smi"])
logging.info(hostname.decode("utf-8"))
logging.info(nvidia_smi.decode("utf-8"))

nstlim = 500000

sim = Simulation()

# Minimization
sim.executable = "pmemd.cuda"
sim.topology = "smirnoff.prmtop"
sim.prefix = "minimize"
sim.inpcrd = "smirnoff.inpcrd"
sim.path = "./"
sim.ref = "smirnoff.inpcrd"
sim.config_pbc_min()
sim.cntrl["maxcyc"] = 500
sim.cntrl["ncyc"] = 400
sim.cntrl["ntr"] = 1
sim.cntrl["restraint_wt"] = 50.0
sim.cntrl["restraintmask"] = "'@Pb'"
sim.cntrl["cut"] = 9.0
sim.restraint_file = "disang.rest"
sim.run(fail_ok=False)

# Equilibration
sim.config_pbc_md()
sim.executable = "pmemd.cuda"
sim.topology = "smirnoff.prmtop"
sim.path = "./"
sim.restraint_file = "disang.rest"

sim.cntrl["nstlim"] = nstlim
sim.cntrl["ntwx"] = 250
sim.cntrl["ntwr"] = 250
sim.cntrl["cut"] = 9.0

iteration = 0
sim.prefix = "equil.{:03d}".format(iteration)
sim.inpcrd = "minimize.rst7"
sim.ref = "smirnoff.inpcrd"

while not os.path.isfile("equil.rst7") and iteration < 10:
    sim.run(fail_ok=True)
    with open("equil.{:03d}.out".format(iteration)) as f:
        for line in f.readlines():
            if re.search(" TIMINGS", line):
                shutil.copy(sim.restart, "equil.rst7")

    recenter("smirnoff.prmtop", "equil.{:03d}.rst7", mask="@Pb")
    iteration += 1
    sim.prefix = "equil.{:03d}".format(iteration)
    sim.inpcrd = "equil.{:03d}.rst7".format(iteration - 1)
    sim.ref = "smirnoff.inpcrd"


recenter("smirnoff.prmtop", "equil.rst7", mask="@Pb")
recenter("smirnoff.prmtop", "smirnoff.inpcrd", mask="@Pb")

# Production
for iteration in range(0, 10):
    sim.executable = "pmemd.cuda"
    sim.path = "./"
    sim.topology = "smirnoff.prmtop"
    sim.restraint_file = "disang.rest"
    sim.config_pbc_md()

    sim.prefix = "prod.{:03d}".format(iteration)
    if iteration == 0:
        sim.inpcrd = "equil-centered.inpcrd"
    else:
        sim.inpcrd = "prod.{:03d}.rst7".format(iteration - 1)

    sim.ref = "smirnoff-centered.inpcrd"
    sim.cntrl["ntx"] = 5
    sim.cntrl["irest"] = 1
    sim.cntrl["nstlim"] = nstlim
    sim.cntrl["ntwr"] = nstlim
    sim.cntrl["ntwx"] = 250
    sim.cntrl["ntxo"] = 2
    sim.cntrl["cut"] = 9.0
    sim.cntrl["iwrap"] = 0

    sim.restraint_file = "disang.rest"
    sim.run(fail_ok=False)

```